<a href="https://colab.research.google.com/github/ssanchezgoe/curso_deep_learning_economia/blob/main/NBs_Google_Colab/S10_Optimizadores_Retroproyeccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Curso Deep Learning: Economía</h1>

## S10: Optimizadores y algoritmo de retroproyección.

# Optimizadores



Vimos que la técnica de propagación hacia atrás utiliza el gradiente de descenso para ealizar la optimización de los pesos en cada pasada.

La actualización mediante el gradiente de descenso lo hemos definico como:

$$w_{t}=w_{t-1}-\eta  \nabla C(w)$$

Donde $\eta$ es llamado la taza de aprendizaje, éste debe escogerse de tal forma que no sea tan pequeño como para hacer muy lenta la convergencia, ni muy grande como para hacer que nuestros pesos diverjan.

Uno de los  mayores incovenientes con éste algoritmo es que en el caso de las funciones de coste no convexas es probable quedar atrapado en un mínimo local, y jamás llegar a un mínimo global, para mitigar dicha posibilidad se han creado algunas variaciones como el gradiente de descenso estocástico o el gradiente de descenso por minibatch.

### Gradiente de descenso estocástico.

Como es planteado el gradiente de descenso solo hace un recalculo de los parámetros del modelo una vez ha hecho un paso completo de todos los ejemplos, ésto hace que el algorítmo sea de lenta convergencia y en caso de tener muchos ejemplos de entrenamiento pueda desbordar la memoria de la máquina.

En el gradiente estocástico se realiza una actualización cada que se ha entregado un ejemplo a la red, con eésto se consigue una mayor varianza y por tanto las actualizaciones fluctuan en intensidad y es más difícil que el algoritmo quede atrapado en un mínimo local si éste no es bastante "profundo" lo cual garantiza mínimos más estables.

### Gradiente de descenso por minibatch.

Una de las desventajas del SDG reside en que al hacer una actualización por ejemplo puede ser bastante lento, una solución intermedia es pasar unos cuantos ejemplos antes de hacer la actualización (entregar un batch), con ello aumentamos la velocidad de entrenamiento pero no perdemos la ventaja de generar varianza sobre las actualizaciones para evitar ser atrabados en un mínimo local.

Todos éstos métodos utilizan la mísma ecuación y la mísma idea de fondo, se diferencian en la cantidad de ejemplos pasados por la red antes de realizar la actualización de los pesos.






## Momentum.

Dado que los métodos de descenso de gradiente calculan las variaciones en todas direcciones suelen oscilar de forma innecesaria, lo cual hace que su convergencia sea más lenta. Para solucionar ésto se usa la técnica de momentum, en analogía con el momentum de la física clásica.

Cuando una pelota rudea por una pendiente su velocidad aumenta en la dirección de movimiento (su derivada), mientras que en las demás direcciones no lo hace. En el método se sigue la misma idea, la actualización se hará en las direcciones en las que el gradiente aumentó más la actualización pasada, así nos aseguramos de no actualizar (demasiado) los pesos de manera innecesaria en direcciones que no contribuyen a la busqueda del mínimo.

Si ahora notamos el paso de actualización de los pesos como: $$w_t = w_{t-1}-V(t)$$

El $V$ en la iteración $t$ estará dado por:

$$V(t) = \gamma V(t-1)+\eta \nabla (w)$$

Ahora tenémos un hiperparámetro nuevo $\gamma$ que se encargará de determinar qué tanto influencia la actualización pasada a la actual, generalmente será un número al rededor de $0.9$

## Gradiente acelerado de Nesterov.

En los años 80 el investigador Yurii Nesterov se dio cuenta que el momentum tenía un problema, al igual que en la analogía de la pelota, al llegar al mínimo el momento remanente empuja la pelota un poco más arriba del mínimo, también pasará lo mismo en el caso del error de la red, ésto puede demorar la convergencia o incluso hacer que el error no converja nunca. 

Para solucionar dícho problema Nasterov propuso no solo usar la influencia de la actualización pasada en la actualización presente si no que también se tendría en cuenta para el cálculo del gradiente (similar a tener en cuenta la aceleración para determinar que tan cerca se está de un cambio de direción).

Así tenemos que:

$$V(t) = \gamma V(t-1)+\eta \nabla (w- \gamma V(t-1))$$

Por tanto la actualización disminuirá su "momentum" al acercarce al mínimo y por tanto no se alejará mucho de él.

## Adagrad y Adadelta.

Adagrad (adaptative gradient) nos permite actualizar de manera adaptativa cada parámetro, haciendo grandes modificaciones a parámetros infrecuentes mientras que lo hará con más cuidado a parámetros frecuentes. Ésto será útil cuando tratemos con datos dispersos (sparse data). 

Cada parámetro $w_i$ será actualizado así:

$$w_{i,t} = w_{i,t-1}-\frac{\eta}{\sqrt{G_{ii}+\epsilon}}\frac{\partial C }{\partial  w_i}$$

El termino $G_{ii}$ es la acumulación de los gradientes pasados, $\epsilon$ es un número pequeño para evitar la divergencia al dividir por cero.

Un problema con ésta técnica es el hecho de que al acumular los gradientes en el denominador poco a poco el gradiente se hace más y más suave (también sufre de una especie de desvanecimiento) además de que el tiempo de entrenamiento también se hará mayor pues los pasos serán cada vez menores.

Para atacar el problema del desvanecimiento usaremos **Adadelta**, en éste caso no tomaremos toda la acumulación de gradientes si no que se hará con una ventana de tamaño $\rho$, definida por nosotros, con ello tenemos que el denominador será el RMS (sobre la vetana) de los gradientes, así:
$$w_{i,t} = w_{i,t-1}-\frac{\eta}{RMS(\frac{\partial C }{\partial  w_i})}\frac{\partial C }{\partial  w_i}$$

Ambas técnicas tienen la ventaja de no ser necesaria la definición de una razón de aprendizaje, ya que de manera iterativa ésta será calculada.

## Adam (Adaptative moment estimator).

Adam calculará tanto el promedio de los gradientes pasados  (como Adadelta), así como el promedio del decaimiento de los graientes pasados (como momentum).

Por tanto Adam es un estimador de momentos estadísticos adaptatico (en cada iteración). 

El primer momento acumulado será:
$$\hat{m} =\frac{m_t}{1-\beta_1^t}$$

y el segundo:
$$\hat{v} =\frac{v_t}{1-\beta_2^t}$$

y la actualización de los pesos será adaptada como:
$$w_{i,t} = w_{i,t-1}-\frac{\eta}{\hat{v}+\epsilon}\hat{m}$$

$\beta_i$ serpan hiperparámetros del optimizador.

Adam es uno de los mejores optimizadores ya que es rápido, evita las fluctuaciones exageradas de los parámetros en direcciones no convenientes, y el desvanecimiento de los gradientes.


![](https://miro.medium.com/max/620/1*XVFmo9NxLnwDr3SxzKy-rA.gif)

![](https://miro.medium.com/max/620/1*SjtKOauOXFVjWRR7iCtHiA.gif)

# Optimizadores en Keras.

Como sabemos, uno de los pasos fundamentales en Keras para definir y entrenar un modelo es compilarlo, allí debemos decidir cual será el método de optimización a usar, además de la función de perdida y las métricas a usar.

Los diferéntes optimizadores se importan como `from keras import optimizers` [https://keras.io/optimizers/](https://keras.io/optimizers/)

Allí encontraremos los anteriores (y unos cuantos más).

Podemos llamarlos en el compilador de dos maneras:



*   Instanciandolos antes del compilador. 
*   Llamandolos dentro del compilador.
Veamos:


In [ ]:
from tensorflow.keras import optimizers

model = Sequential()
model.add(Dense(64, kernel_initializer='uniform', input_shape=(10,),activation='softmax'))
#Instanciamos previo a llamarlo en el compilador, en éste caso podemos cambiar los parámetros del optimizador
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [ ]:
#dando su nombre como parámetro al compilador, en éste caso los parámetros del optimizador usados serán sus valores por defecto
model.compile(loss='mean_squared_error', optimizer='sgd')

In [ ]:
#Para llamar un gradiente estocástico
#Recuerden que la difencia estará en cómo pasemos los bathc, además podemos darle un momentum o un nesterov,
#SDG contiene entonces los optimizadores SDG, momentum, y Nesterov en una sola clase
keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False)

In [ ]:
#Adagrad se llamará como:
#el learning_rate es el valor inicial de dicho parámetro, 
#recuerde que éste método es adaptativo y éste valor cambiará con las iteraciones
keras.optimizers.Adagrad(learning_rate=0.01)

In [ ]:
#Adadelta será:
#El parámetro $\rho$ controla la fracción de gradientes a tener en cuenta en cada paso de optimización.
keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)

In [ ]:
#finalmente Adam
#donde amsgrad determina si se aplica una modificaión al método o no basada en https://openreview.net/forum?id=ryQu7f-RZ
keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

Recuerde que cada optimizador tendrá sus ventajas o desventajas dependiendo del problema, la función de coste y nuestro poder computacional.
Por lo general el usdo de Adam o SDG con momentum serán una buena elección como primera prueba.

Finalmete, en todos los optimizadores es posible usar los parámetros clipnorm (todos los valores de los gradientes serán recortados a una norma máxima) y clipvalue (todos los valores de los gradientes serán recortados a un valor mínimo de -n y máximo de n)

In [ ]:
#los valores de los gradientes estarán como máximo en una norma de 1
sgd = optimizers.SGD(lr=0.01, clipnorm=1.)
#los valores de los gradientes estarán mínimo en -0.5 y máximo en 0.5
sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)

# Propagación hacia atrás (Back propagation)

Aunque la propagacación hacia atrás se haga de manera automática en la mayoría de los paquetes computacionales es bueno que nos demos una idea de cómo funciona y su importancia dentro del DL.

Éste método fue introducido a mediados de los años 70, pero no fue hasta el año 86, cuando un [artículo](http://www.nature.com/nature/journal/v323/n6088/pdf/323533a0.pdf) publicado en la revista Nature por David Rumelhart, Geoffrey Hinton, and Ronald Williams; mostrando cómo su implementación en diferentes redes neuronales aumentava su velocidad de entrenamiento respecto a las técnicas usadas en la época, haciendo que problemas intratables hasta el momento se pudieran resolver con relativa fácilidad. Ésto hizo que la propagación hacia atrás se popularizara y que hasta ahora sea la técnica más usada en el entrenamiento de redes neuronales.

La dificultad de hacer la actualización de los pesos de una red profunda radica en el hecho de que cada peso influye en todos los pesos de la capa subsiguiente (y por tanto en todos los demás de forma indirécta), ésto nos lleva a que sea una tarea computacionalmente costosa hacer una actualización de los pesos hacia delante pues una pequeña modificación hacia adelante puede tener consecuencias gigantes en las capas sucesivas.

La propagación hacia atrás nos propone usar un método como el gradiente de descenso (iniciando con el error en la capa de salida) e ir propagando los errores hacia atrás como si cada capa fuera la salida de la anterior. Con ello podemos actualizar poco a poco y teniendo en cuenta los errores acumulados por la red. De forma efectiva estámos actualizando los pesos con toda la información disponible sin el riesgo de un efecto descontrolado en los pesos (como sucede en la propagación hacia adelante).

Son 4 las ecuaciones fundamentales del método:

Denotaremos como $L$ la última capa, $l$ las capas ocultas, $j$ una neurona de la capa $l$, $\sigma$ las funciones de activación de las capas ocultas y $a_j$ es la salida $j$ de la red ($a$ sería el vector de todas las salidas predichas por la red) y $z_j^l$ es la entrada pesada a la función de activación de la capa $l$ en la neurona $j$.




*   Una ecuación para el error de la capa de salida:
$$\delta_j ^L = \frac{\partial C}{\partial a_j}\sigma(z_j^L)$$
*   Una ecuación para el error de una capa ocualta dada por el error de la capa siguiente (error propagado hacia atráss):
$$\delta^l = ((w^{l+1})^T \delta^{l+1})\sigma(z^l)$$
*   Una ecuación para la derivada de la función de costo para cualquier bias en la red:
$$\frac{\partial C}{\partial b_j^l}= \delta_j^l$$
*    Una ecuación para la derivada parial de la función de costo respecto a cualquier peso de la red:
$$\frac{\partial C}{\partial{w_{jk}^l}}=a_k^{l-1}\delta_j^l$$


Con las dos primeras ecuaciones nos es posible calcular el error en cualquier neurona, partiendo desde la capa de salida con la primer ecuación y llendo hacia atrás (propagando el error) cuantas veces sea necesario con la segunda ecuación.

Con ello y las dos últimas es posible calcular las derivadas del costo en cualquier punto de la red y aplicando el gradiente de descenso tendríamos nuestra actualización de pesos de una forma más rápida y eficiente que con el uso de inversiones matriciales.


Con las ecuaciones a mano es fácil imaginar el algoritmo necesario para realiazar la propagación hacia atrás:
*  Cree la activación para la capa de entrada $a_1$.
*  Propague hacia adelanta para cada $l= 2,3,...,L$ calculando $z^l = w^la^{l-1}+b^l$ y $\sigma(z^l)$
*   Calcule el error de la capa de salida $\delta^L$
*   Propague hacia atrás para cada $l=l-1, l-2...2$ calculando el correspondiente $\delta^l$
*  Calcule los gradientes correspondientes a cada peso y cada bias y aplique el método de descenso de gradiente para actualizar cada peso.

Si bien ésta es algorítmo detrás del método no es necesario aprender los detalles o las ecuaciones, pues éste ya viene integrado en las bibliotecas que usamos para realizar machine learning como Scikitlearn o Keras, sin embargo si es importante comprender su uso y porqué es tan importante y usado por los científicos de datos.